# LCM2023 - Poster session, 5-8 September 2023
## Retrofitting of Italian vernacular built stock with bio-based building materials: a viable solution to mitigate climate change effects?
Example of the brightway code used to compute the LCA results included in the contribution to the LCM conference.

In [1]:
Project_name="BIOED" 
ei_version="3.9.1"
ei_type="cut-off"
ei_name="Ecoinvent "+ei_version+" "+ei_type+" "+Project_name
ei_name

'Ecoinvent 3.9.1 cut-off BIOED'

## Library used

In [2]:
# brightway 25
import bw2analyzer as ba
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import matrix_utils as mu
import bw_processing as bp
# import bw_temporalis

import pandas as pd
import os 
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px

path_name = r'C:\Users\niky2\OneDrive - Politecnico di Milano\Dottorato\Progetti\01-Paper climate\Aggiornamento LCA'
os.chdir(path_name)

In [10]:
bd.projects.set_current('BIOED')
  

In [11]:
bd.projects.current

'BIOED'

In [15]:
bi.bw2setup() # se il project esiste già, questo comando non fa nulla
# se il progetto non esiste, bw2 crea il biosphere database, che è il database dei flussi elementari. Crea anche gli
# LCIA methods di base. Si possono poi importare altri flussi elementari o LCIA methods.
# il biosphere database è creato a partire da file presenti all'interno dei packages di bw2, che vengono richiamati quando serve

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
Vacuuming database 
Vacuuming database 


OperationalError: cannot VACUUM from within a transaction

In [ ]:
list(bd.databases)

## Ecoinvent 3.9.1

In [6]:
if len(bd.databases)==1: # eseguo questo comando solo se ho solo il biosphere database (database dei flussi elementari). Procedo a importare un nuovo database io, a partire da ecoinvent 3.8 cutoff 
    # Nel comando qui sotto, dovrai mettere l'indirizzo del tuo pc in cui hai salvato ecoinvent in formato ecospold. 
    # Attenzione a unzippare il file ecospold e ad arrivare alla cartella datasets nell'indirizzo
    fpei = r'D:\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets'
    # i comandi qui sotto servono a elaborare il database e infine "scriverlo" nel project
    ei = bi.SingleOutputEcospold2Importer(fpei, ei_name) # creazione dell'oggetto 'Ecoinvent 3.8 cut-off OWC'.
    ei.apply_strategies() # elaborazione del database
    ei.statistics() # statistiche finali (eventuali errori vengono mostrati qui)
    ei.write_database() # scrittura del database

Extracting XML data from 21238 datasets
Extracted 21238 datasets in 80.33 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:59


Title: Writing activities to SQLite3 database:
  Started: 06/06/2023 09:53:59
  Finished: 06/06/2023 09:54:58
  Total time elapsed: 00:00:59
  CPU %: 96.50
  Memory %: 48.85
Created database: Ecoinvent 3.9.1 cut-off BIOED


## Implemented function:

In [4]:
def new_act_diff_location(database, activity_id, locations, new_name, project_name=Project_name):
    ''' This function will create a copy of an activity, identified with activity.id, and will modify the copied activity by 
    substituting the technosphere processes with the preferable location indicated by the parameter locations.
    '''
    # is the modified process already in the database?
    if (len([act for act in database if act['name']==new_name])==0):
        # find activity to modify
        act_origin = [act for act in database if act.id==activity_id][0]
        act_new = act_origin.copy()
        act_new['name'] = new_name
        act_new['location'] = project_name
        act_new.save()
        
        for exc in act_new.technosphere():
            exc.delete()
            exc.save()
            act_new.save()
            
        # If GLO or RoW were not considered, add it at the end
        if 'RoW' not in locations:
            locations.append('RoW')
        if 'GLO' not in locations:
            locations.append('GLO')
        
        locations.reverse()
        
        for exc in act_origin.technosphere():
            for loc in locations:
            # Search for an exact match of the reference product with specified location
                for activity in database:
                    if activity['reference product'] == exc['name'] and loc in activity['location'] and activity['name'] == exc.input['name']:
                        new_edge = activity
                
             
            act_new.new_edge(
                            input=new_edge.key,
                            name=new_edge['name'],
                            amount=exc.amount,
                            type="technosphere",
                        ).save()
            new_edge.save()             
        
        return act_new
    
    print('The modified activity is already present in the database!!')
    return [act for act in database if act['name']==new_name][0]
    

In [3]:
def new_activity(database, activity_name, activity_code, activity_unit, activity_reference, exchanges, project_name=Project_name, bio = bd.Database('biosphere3')):
    '''This function will create a new activity in the given database, adding in input the given exchanges'''
    
    # is the modified process already in the database?
    if (len([act for act in database if act['name']==activity_name and act['location']==project_name])==0):
        print('####################################################################################################')
        print(f'Creating the activity: {activity_name}....')
        proj_Database=database.new_node(code=activity_code,
                                                         name=activity_name, # nome
                                                         location=project_name, 
                                                         type='process', 
                                                         unit=activity_unit) # unità di misura del reference flow del processo
        # e lo salvo poi nel database (bisogna sempre ricordarsi di salvare, altrimenti le modifiche)
        proj_Database.save()
        # bw2 usa il seguente vocabolario, rispetto a OpenLCA: activity=process e exchange=flow
        # Qui sotto cerco uno specifico processo (activity), nel database 'ecoinvent 3.6 LCE' indicando che 'Polymer transport - LCE' sia nel nome di tale processo.
        # Ovvero, sto cercando il processo che ho appena creato nelle righe sopra.
        new_act=[act for act in database if act['name']==activity_name and act['location']==project_name][0]
        # Aggiungo il nome del reference flow nelle info di questo processo
        new_act['reference product']=activity_reference
        # e poi salvo.
        new_act.save()
        
        print(f'Inseriting the inputs of {activity_name}:')
        for j in range(len(exchanges)):
            if exchanges['code'][j]==new_act['code']:
                if exchanges['location'][j] == 'biosphere':
                    new_input=[ef for ef in bio if exchanges['process'][j] in ef['name']][0]
                    
                    new_exc=new_act.new_edge(input=new_input.key, # Questo è il provider
                               amount=exchanges['amount'][j], # quantità
                               type='biosphere')
                    new_exc.save()
                    new_act.save()
                    print(f"Edge: {new_input['name']};    Amount: {exchanges['amount'][j]}")
                    
                else: 
                    new_input=[act for act in ei if act['name'] == exchanges['process'][j]
                                if exchanges['location'][j] in act['location']][0]

                    new_exc=new_act.new_edge(input=new_input.key, # Questo è il provider
                                   amount=exchanges['amount'][j], # quantità
                                   unit=new_input["unit"], # unità di misura
                                   name=new_input['reference product'], # nome del flusso
                                   type='technosphere')
                    new_exc.save()
                    new_act.save()        
                    print(f"Edge: {new_input['reference product']};    Amount: {exchanges['amount'][j]}")
        
        
    else:
        print('______________________________________________________________________________________________________')
        print(f'The activity: {activity_name} is already present in the database, no need to create a new one!!')
    
    

## Materials LCI

In [7]:
ei = bd.Database(ei_name)

In [8]:
bio = bd.Database('biosphere3')

In [8]:
[act for act in ei if 'BIOED' in act['location']]

['earten plaster B2 production' (kilogram, BIOED, None),
 'glass wool mat production, IT' (kilogram, BIOED, None),
 'natural hydraulic lime fibre-reinforced screed production' (kilogram, BIOED, None),
 'Rasocalce production' (kilogram, BIOED, None),
 'Decorcalce production' (kilogram, BIOED, None),
 'lime mortar production, IT' (kilogram, BIOED, None),
 'earten plaster B1 production' (kilogram, BIOED, None),
 'expanded perlite production, IT' (kilogram, BIOED, None),
 'stone wool production, packed, IT' (kilogram, BIOED, None),
 'lean concrete production, with cement CEM II/A, IT' (cubic meter, BIOED, None),
 'earten plaster R1 production' (kilogram, BIOED, None),
 'igloo production' (kilogram, BIOED, None),
 'gypsum fibreboard production, IT' (kilogram, BIOED, None),
 'ceramic tile production, IT' (kilogram, BIOED, None),
 'clay plaster production, IT' (kilogram, BIOED, None),
 'gypsum plasterboard production, IT' (kilogram, BIOED, None),
 'cork granulate production' (kilogram, BIOED,

## creation of proxy activities for Italy

### lime mortar production, IT

for example to create the activity 'lime mortar production' located in Italy, we started from the activity located in Switzerland and chainging the input with the corresponding activities, coherently selected for the Italian case.  
To search the locations, we considered the presence of activities with the following hierarchical order:  
- IT;
- Europe without Switzerland;
- RER;
- RoW;
- GLO.

Creation of the activity 'lime mortar production, IT' from the correspondent activity in Switzerland but with different location for the input.

In [9]:
lime_mortar_CH = [act for act in ei if 'lime mortar production' in act['name'] and 'CH' in act['location']][0]
lime_mortar_CH.id

IndexError: list index out of range

In [43]:
LM_IT_node = new_act_diff_location(ei, lime_mortar_CH.id, ['IT', 'Europe', 'RER'], "lime mortar production, IT", Project_name)

In [46]:
[exc for exc in LM_IT_node.technosphere()]

[Exchange: 0.56 kilogram 'market for cement, Portland' (kilogram, Europe without Switzerland, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 0.0023 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 3.33e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 0.0278 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, IT, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 6.67e-06 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RER, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 0.043 kilogram 'market for lime, hydrated, packed' (kilogram, RER, None) to 'lime mortar production, IT' (kilogram, BIOED, None)>,
 Exchange: 0.105 kilogram 'market for lime, hydraulic' (kilogram, RER, None) to 'lime mortar production, IT' (kilo

In [47]:
[exc for exc in lime_mortar_CH.technosphere()]

[Exchange: 0.56 kilogram 'market for cement, Portland' (kilogram, CH, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 0.0023 kilogram 'market for chemical, organic' (kilogram, GLO, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 3.33e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 0.0278 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, CH, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 6.67e-06 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RER, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 0.043 kilogram 'market for lime, hydrated, packed' (kilogram, RER, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 0.105 kilogram 'market for lime, hydraulic' (kilogram, RER, None) to 'lime mortar production' (kilogram, CH, None)>,
 Exchange: 1.0 kilogram 'packing, cement' (kilogram,

### plaster mixing, IT 

In [44]:
plaster_mixing_CH = [act for act in ei if 'plaster mixing' in act['name'] and 'CH' in act['location']][0]

In [31]:
plaster_mixing_CH.id

9720

In [78]:
PM_IT_node.delete()

In [79]:
PM_IT_node = new_act_diff_location(ei, plaster_mixing_CH.id, ['IT', 'Europe', 'RER'], "plaster mixing, IT", Project_name)

In [80]:
[exc for exc in PM_IT_node.technosphere()]

[Exchange: 0.08 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, IT, None) to 'plaster mixing, IT' (kilogram, BIOED, None)>,
 Exchange: 0.0002 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RER, None) to 'plaster mixing, IT' (kilogram, BIOED, None)>]

## Materials LCI from BIOED_import.xlsx

In [15]:
activities_df = pd.read_excel(r"BIOED_import.xlsx", "act")
activities_df

,code,unit,name,reference product,location
0,BIOED_1,kilogram,natural hydraulic lime fibre-reinforced screed...,natural hydraulic lime fibre-reinforced screed,BIOED
1,BIOED_2,kilogram,Rasocalce production,Rasocalce,BIOED
2,BIOED_3,kilogram,igloo production,igloo,BIOED
3,BIOED_4,kilogram,Decorcalce production,Decorcalce,BIOED
4,BIOED_5,kilogram,earten plaster B1 production,earten plaster B1,BIOED
5,BIOED_6,kilogram,earten plaster B2 production,earten plaster B2,BIOED
6,BIOED_7,kilogram,earten plaster R1 production,earten plaster R1,BIOED
7,BIOED_8,kilogram,cork granulate production,cork granulate,BIOED


In [14]:
exchanges_df = pd.read_excel(r"BIOED_import.xlsx", "exc")
exchanges_df

,amount,unit,process,location,code,activity
0,0.833333,kilogram,"lime mortar production, IT",BIOED,BIOED_1,natural hydraulic lime fibre-reinforced screed...
1,0.166667,kilogram,market for tap water,Europe without Switzerland,BIOED_1,natural hydraulic lime fibre-reinforced screed...
2,1.000000,kilogram,"plaster mixing, IT",BIOED,BIOED_1,natural hydraulic lime fibre-reinforced screed...
3,0.208333,kilogram,"lime production, hydraulic, IT",BIOED,BIOED_2,Rasocalce production
4,0.625000,kilogram,market for silica sand,GLO,BIOED_2,Rasocalce production
5,0.166667,kilogram,market for tap water,Europe without Switzerland,BIOED_2,Rasocalce production
6,1.006000,kilogram,"polypropylene production, granulate",RER,BIOED_3,igloo production
7,1.006000,kilogram,injection moulding,RER,BIOED_3,igloo production
8,0.250000,kilogram,"market for lime, hydrated, packed",RER,BIOED_4,Decorcalce production
9,0.166667,kilogram,market for tap water,Europe without Switzerland,BIOED_4,Decorcalce production


In [32]:
for i in range(len(activities_df)):
    new_activity(ei, activities_df['name'][i], activities_df['code'][i], 
                 activities_df['unit'][i], activities_df['reference product'][i], exchanges_df)

______________________________________________________________________________________________________
The activity: natural hydraulic lime fibre-reinforced screed production is already present in the database, no need to create a new one!!
______________________________________________________________________________________________________
The activity: Rasocalce production is already present in the database, no need to create a new one!!
______________________________________________________________________________________________________
The activity: igloo production is already present in the database, no need to create a new one!!
####################################################################################################
Creating the activity: Decorcalce production....
Inseriting the inputs of Decorcalce production:
Edge: lime, hydrated, packed;    Amount: 0.25
Edge: tap water;    Amount: 0.16666666666666669
Edge: silica sand;    Amount: 0.5833333333333334
______________